In [141]:
import pandas as pd
df = pd.read_excel("Test O_G_Equipment_Data.xlsx")

df.to_csv("data.csv", index=False)

## Major task

 - Investigate one piece of equipment in different time cycles to understand what **characteristics and parameters** of the sensors **might indicate that the equipment is on the verge of failing**.

## Task 1 

- Calculate how many times the equipment has failed.

During the FPSO’s operation, various factors can cause the machine to fail and prolong its failure state. 

We ask you to explore the available data, identify, and calculate the number of times the
equipment has failed throughout its operation.

In [271]:
import plotly.express as px

fig = px.bar(df["Fail"].value_counts().reset_index(), x='Fail', y='count', text_auto='.s')
fig.update_traces(textposition="outside")
fig.update_layout(
        title=dict(
            text="Count of Fail and Not Fail Events",
            x=0.5, xanchor='center',
            y=0.94, yanchor='top',
            font=dict(size=26),
            pad=dict(t=10, b=10)
        ),
        margin=dict(l=60, r=40, t=120, b=50)
    )
fig.update_yaxes(range=[0, df.shape[0]])
fig.show()

In [268]:
"{:.2%}".format(df['Fail'].mean())

'8.25%'

## Task 1 answer

- From figure Count of *Fail and Not Fail Events* we may answer that the equipment has failed 66 times

- That is 8.25 % from all observed Cycles.

## Task 2

- Categorize equipment failures by setup configurations (Preset 1 and Preset 2).

- How do the variables Preset_1 and Preset_2 behave during operation? 

- What insights can we derive from these variables?

In [262]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def failure_report(data, group):
    data_copy = data.copy()

    total = (data_copy[group].astype(str)
            .value_counts().rename_axis(group)
            .reset_index(name='Number of observations'))

    taxa = (
        data_copy[data_copy["Fail"]][group].astype(str).value_counts() 
        / data_copy[group].astype(str).value_counts()
        ).reset_index(name='Failure percentage')

    base = (total.merge(taxa, on=group, how='left')
                .fillna({'Failure percentage': 0})
                .sort_values(group))

    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(
        go.Bar(x=base[group], y=base['Number of observations'],
            name='Number of observations',
            text=base['Number of observations'], textposition='outside'),
        secondary_y=False
    )

    fig.add_trace(
        go.Scatter(x=base[group], y=base['Failure percentage'],
                name='Failure percentage',
                mode='lines+markers+text',
                line=dict(color='red', width=2),
                text=base['Failure percentage'],
                texttemplate='%{y:.1%}',
                textposition='top center',
                cliponaxis=False),
        secondary_y=True
    )

    fig.update_yaxes(title_text='Number of observations', secondary_y=False)
    fig.update_yaxes(title_text='Failure (%)', tickformat='.1%', secondary_y=True)
    fig.update_xaxes(title_text=group)
    fig.update_layout(
        title=dict(
            text=f"Number of observations versus Failure percentage<br><span style='font-size:16px;'>Considering {group}</span>",
            x=0.5, xanchor='center',
            y=0.94, yanchor='top',
            font=dict(size=26),
            pad=dict(t=10, b=10)
        ),
        margin=dict(l=60, r=40, t=120, b=50)
    )
    fig.show()

def heatmap_failure(data):

    data_copy = data.copy()

    matrix_data = (data_copy.groupby(['Preset_1','Preset_2'])['Fail'].mean()
        .mul(100).unstack(fill_value=0).sort_index())

    matrix_data.columns = matrix_data.columns.astype(str)
    matrix_data.index = matrix_data.index.astype(str)

    fig = px.imshow(matrix_data, text_auto='.1f', aspect='auto',
            color_continuous_scale='Reds',
            labels=dict(color='Fail %')).update_layout(
        xaxis_title='Preset_2', yaxis_title='Preset_1'
    )

    fig.update_layout(
        title=dict(
            text="Heatmap<br><span style='font-size:16px;'>Combinating Preset_1 and Preset_2</span>",
            x=0.5, xanchor='center',
            y=0.94, yanchor='top',
            font=dict(size=26),
            pad=dict(t=10, b=10)
        ),
        margin=dict(l=60, r=40, t=120, b=50)
    )
    
    
    fig.show()

In [ ]:
### Changes from presets

d = (df[['Cycle','Preset_1']]
       .sort_values('Cycle')
       .assign(prev=lambda x: x['Preset_1'].shift()))

modifications = d.loc[d['Preset_1'].ne(d['prev']) & d['prev'].notna(), ['prev','Preset_1']]

transitions = (modifications.astype(int)
                .value_counts()
                .reset_index(name='Times')
                .rename(columns={'prev':'From','Preset_1':'To'})
                .sort_values('Times', ascending=False))

print(transitions)

   From  To  Times
0     1   2     97
1     2   1     90
2     2   3     90
3     3   1     87
4     3   2     83
5     1   3     80


In [ ]:
### Changes from presets

d = (df[['Cycle','Preset_2']]
       .sort_values('Cycle')
       .assign(prev=lambda x: x['Preset_2'].shift()))

modifications = d.loc[d['Preset_2'].ne(d['prev']) & d['prev'].notna(), ['prev','Preset_2']]

transitions = (modifications.astype(int)
                .value_counts()
                .reset_index(name='Times')
                .rename(columns={'prev':'From','Preset_2':'To'})
                .sort_values('Times', ascending=False))

print(transitions)

    From  To  Times
0      6   2     20
1      3   4     18
2      4   1     16
3      2   1     16
4      2   3     16
5      7   6     16
6      5   7     16
7      4   6     15
8      8   6     15
9      1   7     15
10     7   2     15
11     6   4     15
16     7   5     14
19     8   5     14
18     8   3     14
17     1   8     14
12     4   7     14
15     8   7     14
14     5   3     14
13     5   2     14
30     2   4     13
29     2   8     13
25     3   8     13
27     1   6     13
26     3   7     13
28     3   6     13
24     1   2     13
22     4   5     13
21     5   8     13
20     6   1     13
23     7   1     13
33     2   5     12
34     6   5     12
35     7   8     12
32     6   3     12
31     3   5     12
36     6   8     11
37     7   4     11
38     5   4     11
39     8   4     11
45     4   2     10
48     4   8     10
47     1   3     10
46     5   6     10
41     1   5     10
44     2   6     10
43     6   7     10
42     4   3     10
40     7   3     10


In [ ]:
### Changes from presets

aux_data = df.copy()
aux_data["Preset_1_2"] = df['Preset_1'].astype(str) + df['Preset_2'].astype(str)

d = (aux_data[['Cycle','Preset_1_2']]
       .sort_values('Cycle')
       .assign(prev=lambda x: x['Preset_1_2'].shift()))

modifications = d.loc[d['Preset_1_2'].ne(d['prev']) & d['prev'].notna(), ['prev','Preset_1_2']]

transitions = (modifications.astype(int)
                .value_counts()
                .reset_index(name='Times')
                .rename(columns={'prev':'From','Preset_1_2':'To'})
                .sort_values('Times', ascending=False))

print(transitions)

     From  To  Times
0      24  22      6
2      22  38      5
3      31  13      5
4      28  27      5
5      35  38      5
..    ...  ..    ...
290    11  16      1
291    38  33      1
292    38  34      1
293    11  14      1
424    22  26      1

[425 rows x 3 columns]


In [263]:
failure_report(df, "Preset_1")
failure_report(df, "Preset_2")

aux_data = df.copy()
aux_data["Preset_1_2"] = df['Preset_1'].astype(str) + df['Preset_2'].astype(str)

failure_report(aux_data, "Preset_1_2")

heatmap_failure(df)

## Task 2 answer

#### Equipment failure categorization

- Preset_1 - (3 levels)
    - 1 -> 10.2 % failure - (1.24 the overall failure rate of the dataset)
    - 2 -> 7.5 % failure - (0.91 the overall failure rate of the dataset)
    - 3 -> 7.1 % failure - (0.86 the overall failure rate of the dataset)

- Preset_2 - (8 levels)
    - 1 -> 11.6 % failure - (1.41 the overall failure rate of the dataset)
    - 2 -> 8.9 % failure - (1.08 the overall failure rate of the dataset)
    - 3 -> 5.9 % failure - (0.72 the overall failure rate of the dataset)
    - 4 -> 3.2 % failure - (0.39 the overall failure rate of the dataset)
    - 5 -> 12.0 % failure - (1.45 the overall failure rate of the dataset)
    - 6 -> 8.9 % failure - (1.08 the overall failure rate of the dataset)
    - 7 -> 8.3 % failure - (1.01 the overall failure rate of the dataset)
    - 8 -> 7.0 % failure - (0.85 the overall failure rate of the dataset)

- Categories

    - High failure risk: 
        - Preset_1
            - 1
        - Preset_2
            - 1
            - 5
    - Moderate failure risk:
        - Preset_1
            - 2
        - Preset_2
            - 2
            - 6
            - 7
    - Low failure risk:
        - Preset_1
            - 3
        - Preset_2
            - 3
            - 4
            - 8


#### Preset_1 and Preset_2 behavior during operation

 - ans

#### Derived insights from Preset_1 and Preset_2

 - ans

In [293]:
d = (df[['Cycle','Preset_1','Temperature']]
       .sort_values('Cycle')
       .assign(prev_preset=lambda x: x['Preset_1'].shift(),
               prev_temp=lambda x: x['Temperature'].shift()))

# Pegue apenas os pontos onde houve mudança de preset e calcule a variação de temperatura
trans = d.loc[d['prev_preset'].notna() & d['Preset_1'].ne(d['prev_preset'])].copy()
trans['delta_temp'] = trans['Temperature'] - trans['prev_temp']  # pode ser +/-

# Estatísticas por transição (média das variações e variância)
stats = (trans.groupby(['prev_preset','Preset_1'])['delta_temp']
              .agg(media='mean', variancia=lambda s: s.var(ddof=1), n='size')
              .reset_index()
              .rename(columns={'prev_preset':'de','Preset_1':'para'}))

print(stats)                # tabela com: de, para, media, variancia, n
# print(stats.query('de==2 and para==3')) 

    de  para     media    variancia   n
0  1.0     2  2.485079  1076.588848  97
1  1.0     3  3.510266   887.105813  80
2  2.0     1  0.128162  1054.686749  90
3  2.0     3 -4.388638   836.761251  90
4  3.0     1 -5.020946  1021.852616  87
5  3.0     2  2.939854  1332.713328  83


In [297]:
trans[(trans["Preset_1"]==1)&(trans["prev_preset"]==2.0)]

,Cycle,Preset_1,Temperature,prev_preset,prev_temp,delta_temp
5,6,1,58.336085,2.0,39.989054,18.347031
12,13,1,79.806509,2.0,80.662497,-0.855987
24,25,1,29.906236,2.0,83.213908,-53.307672
38,39,1,61.473079,2.0,45.463432,16.009647
40,41,1,70.869794,2.0,81.304992,-10.435198
...,...,...,...,...,...,...
772,773,1,65.426361,2.0,78.202225,-12.775864
774,775,1,44.833887,2.0,48.852912,-4.019025
787,788,1,82.942913,2.0,74.842740,8.100173
792,793,1,127.670754,2.0,84.990752,42.680002


## Task 3

- Categorize equipment failures by their nature/root cause according to parameter readings (temperature, pressure, and others).

- Analyze patterns in these readings that could indicate specific failure types. 

- How do these patterns differ across operational regimes? 

- Provide insights based on your findings.

In [358]:
import plotly.express as px

def cycles_per_variable(data, x_axis, variable):

    data_copy = data.copy()

    fig = px.line(
        data_copy.sort_values([x_axis]),
        x=x_axis, y=variable
    )

    fig.data[0].update(name=variable, showlegend=True, line=dict(color="royalblue", width=2))

    for x in data_copy.loc[data_copy["Fail"], x_axis].unique():
        fig.add_vline(x=x, line_width=2, line_color="rgba(0,0,0,0.5)")

    fig.add_scatter(x=[None], y=[None], mode="lines",
                    line=dict(color="rgba(0,0,0,0.5)", width=1),
                    name="Failure events")

    fig.update_yaxes(range=[0, 1.1 * data_copy[variable].max()])

    fig.update_layout(
        title=dict(
            text=f"{variable} per {x_axis} with Failure events",
            x=0.5, xanchor='center',
            y=0.94, yanchor='top',
            font=dict(size=26),
            pad=dict(t=10, b=10)
        ),
        margin=dict(l=60, r=40, t=120, b=50)
    )

    fig.show()

def failure_3d_plot(data, x_value, y_value, z_value):

    data_copy = data.copy()

    fig = px.scatter_3d(
        data_copy, x=x_value, y=y_value, z=z_value,
        color="Fail",
        color_discrete_map={True: "black", False: "lightgray"}
    )

    fig.update_traces(marker=dict(size=3))


    fig.update_layout(
        title=dict(
            text=f"3d Plot {x_value} versus {y_value} versus {z_value} per Failure event",
            x=0.5, xanchor='center',
            y=0.94, yanchor='top',
            font=dict(size=26),
            pad=dict(t=10, b=10)
        ),
        margin=dict(l=60, r=40, t=120, b=50)
    )
    fig.show()

def failure_2d_plot(data, x_value, y_value):

    data_copy = data.copy()

    fig = px.scatter(
            data_copy,
            x=x_value, y=y_value,
            color="Fail"
        )

    fig.update_layout(
        title=dict(
            text=f"Scatter Plot {x_value} versus {y_value} per Failure event",
            x=0.5, xanchor='center',
            y=0.94, yanchor='top',
            font=dict(size=26),
            pad=dict(t=10, b=10)
        ),
        margin=dict(l=60, r=40, t=120, b=50)
    )

    fig.show()

def plot_scatter(data, x_value, variable, group, target):

    data_copy = data.copy()

    data_copy[group] = data_copy[group].astype(str)

    cats = sorted(data_copy[group].unique())

    fig = px.scatter(
        data_copy[[x_value,variable,group]].sort_values([x_value, group]), 
        x=x_value, y=variable, color=group, category_orders={group: cats})

    for x in data_copy.loc[data_copy[target], x_value].unique():
        fig.add_vline(x=x, line_color='black', line_width=1)

    fig.update_yaxes(range=[0, 1.1 * data_copy[variable].max()])

    fig.update_layout(
        title=dict(
            text=f"Scatter Plot {variable} versus {x_value} with {group} categories per {target} event",
            x=0.5, xanchor='center',
            y=0.94, yanchor='top',
            font=dict(size=26),
            pad=dict(t=10, b=10)
        ),
        margin=dict(l=60, r=40, t=120, b=50)
    )

    fig.show()

In [359]:
for column in df.columns[3:-1]:
    cycles_per_variable(df, "Cycle", column)

aux_data = df.copy()

aux_data["Module Vibration"] = aux_data.apply(lambda x: ((x["VibrationX"]**2) + (x["VibrationY"]**2) + (x["VibrationZ"]**2))**(1/2), axis=1)

cycles_per_variable(aux_data, "Cycle", "Module Vibration")

failure_2d_plot(df, "Temperature", "Pressure")

failure_2d_plot(aux_data, "Module Vibration", "Frequency")

failure_3d_plot(df, "Temperature", "Pressure", "Frequency")

failure_3d_plot(df, "VibrationX", "VibrationY", "VibrationZ")

for column in df.columns[3:-1]:
    plot_scatter(df, "Cycle", column, 'Preset_1', "Fail")

## Task 3 answer

 - ans

## Task 4

- Create a model (or models) using the technique you think is most appropriate and measure its performance.

- Based on the given time-series dataset, which models or techniques are suitable for predicting whether the equipment will fail before it occurs? 

- Additionally, how can the model's performance be tuned and measured for this task?

## Task 4 answer

 - ans

## Task 5

- Analyze variable importance.

- After developing a model, how can we determine which variables had the greatest impact on the
prediction?

## Task 5 answer

 - ans

#### End